<a href="https://colab.research.google.com/github/anandhc6/Assignment-2/blob/main/PartA_Q4%2C5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Unzipping the dataset

!unzip -q nature_12K.zip

In [ ]:
#installing wandb

!pip install wandb --upgrade

     |████████████████████████████████| 1.7 MB 4.9 MB/s 
     |████████████████████████████████| 181 kB 66.3 MB/s 
     |████████████████████████████████| 144 kB 69.7 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=d92be786ee3d6d0c070d3b32b4c1d7797c492708e60ad714728a3e3a80e8af61
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
import wandb
wandb.login()
!wandb login --relogin
wandb.init(project='A2_partA_Q5',entity="anandh", name = 'Q5a')

In [ ]:
# Importing necessary packages

import pathlib
import numpy as np
from PIL import Image
import os
import glob
import random
from keras.models import load_model
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# from wandb.keras import WandbCallback
import matplotlib.pyplot as plt
from collections import defaultdict




In [ ]:
# Data preperation

def datagen(batch_size, augment_data):

  Data_dir=pathlib.Path('inaturalist_12K') 
  # augment_data=False
  train_path = os.path.join(Data_dir, "train")
  test_path = os.path.join(Data_dir, "val")

  if augment_data:
    train_rawdata = ImageDataGenerator(rescale=1./255,
                                      rotation_range=90,
                                      zoom_range=0.2,
                                      shear_range=0.2,
                                      validation_split=0.1,
                                      horizontal_flip=True)
    test_rawdata = ImageDataGenerator(rescale=1./255)

  else:
    train_rawdata = ImageDataGenerator(rescale=1./255, validation_split=0.1)
    test_rawdata = ImageDataGenerator(rescale=1./255)

  train_data = train_rawdata.flow_from_directory(
      train_path, 
      target_size=(256, 256), 
      batch_size=batch_size, 
      subset="training",
      color_mode="rgb",
      class_mode='categorical',
      shuffle=True,
      seed=42
      )
  
  val_data = train_rawdata.flow_from_directory(
      train_path, 
      target_size=(256, 256), 
      batch_size=batch_size, 
      subset="validation",
      color_mode="rgb",
      class_mode='categorical',
      #shuffle=True,
      seed=42
      )
  test_data = test_rawdata.flow_from_directory(
      test_path, 
      target_size=(256, 256), 
      batch_size=batch_size,
      color_mode="rgb",
      class_mode='categorical',
      # shuffle=True,
      seed=42
      )
  
  return  train_data, val_data, test_data